Este código em como objetivo de realizar a segmentação pelo uso do algoritmos de grabCut.
Ao contrário de ser interativa ela é feita sem inteiração usando o formato completo da imagem que foi lhe dado
Importante anotar que o processamento é bastante demorado.


In [ ]:
import os
import cv2
import h5py
import random
import numpy as np
from skimage import img_as_float32
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
class SegmentationProcessing(BaseEstimator,TransformerMixin):
    
    def __init__(self, path_in, path_out, attributes):
        self.path_in = path_in
        self.path_out = path_out
        self.attributes = attributes
        self.dados_X = []
        self.dados_y = []
        self.x_init = None
        self.y_init = None
        self.image_original = None
        self.image_copy = None
        self.contador = 0
        self.rect = None
        self.attr_moment = ""
    
    def fit(seft,*_):
        return self
    
    
    def get_image_position(self,*_):
        gray = cv2.cvtColor(self.image_copy,cv2.COLOR_BGR2GRAY)

        # Find anything not black, i.e. the image
        nz = cv2.findNonZero(gray)

        # Find top (a), bottom (b), left(c) and right(d) edge of image
        a = nz[:,0,0].min()
        b = nz[:,0,0].max()
        c = nz[:,0,1].min()
        d = nz[:,0,1].max()
        
        rect_final = (a,c,b,d) # a=x_init ,c = y_init, b= x_end, d=y_end
        self.rect = rect_final

        
    def run_segmentation(self,*_):
        # Inicializar uma mascara tendo o tamanho da imagens de base
        mask = np.zeros(self.image_original.shape[:2], np.uint8)
        
        #Initialize mask values 0 to 1
        mask[:,:] = 1
        
        # Initialize background and foreground models
        bgdModel = np.zeros((1, 65), np.float64)
        fgdModel = np.zeros((1, 65), np.float64)
        
        # Run Grabcut algorithm
        cv2.grabCut(self.image_original, mask, self.rect, bgdModel,
                    fgdModel, 5, cv2.GC_INIT_WITH_RECT)

        # Extract new mask
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

        # Apply the above mask to the image
        self.image_original = self.image_original*mask2[:, :, np.newaxis]
        
        # Save the image
        cv2.imwrite("{}/{}/{}.jpg".format(self.path_out,self.attr_moment,self.contador),self.image_original)
        self.contador +=1 
    
    def get_files(self, attribute):
        files = sorted([os.path.join(self.path_in, attribute, file)
                        for file in os.listdir(self.path_in + "/"+attribute)
                        if file.endswith('.jpg')])
        random.shuffle(files)
        return files    
    
    def load_image(self,item):
        
        self.image_original = cv2.imread(item)
        self.image_original = cv2.resize(self.image_original, None, fx=0.5, fy=0.5,interpolation=cv2.INTER_CUBIC)
        self.image_copy = self.image_original.copy()
        
        self.get_image_position()
        self.run_segmentation()
  
    def make_sets(self, *_):
        for attribute in self.attributes:
            self.attr_moment = attribute
            
            dados = self.get_files(attribute)
            
            for item in dados:
                image = self.load_image(item)
            self.contador = 0
            
        
    def transform(self,*_):
        self.make_sets()
        

In [ ]:
path_in = "Imagens/insectos_in"
path_out = "Imagens/insectos_out"
attributes = ["T"]
crop = SegmentationProcessing (path_in, path_out, attributes)

In [ ]:
_=crop.transform()